In [1]:
%load_ext sql
%sql sqlite:///store.db
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## Задание 1
Для каждого артиста вывести все жанры, которые есть в его песнях, и для каждого жанра вывести наиболее продолжительную песню этого артиста.

In [10]:
%%sql
WITH ArtistGenreTracks AS (
    SELECT
        ar.ArtistId,
        ar.Name as ArtistName,
        g.GenreId,
        g.Name as GenreName,
        t.TrackId,
        t.Name as TrackName,
        t.Milliseconds as Duration,
        -- Ранжируем треки по длительности внутри каждого артиста и жанра
        ROW_NUMBER() OVER (
            PARTITION BY ar.ArtistId, g.GenreId
            ORDER BY t.Milliseconds DESC
        ) as duration_rank
    FROM artists ar
    JOIN albums al ON ar.ArtistId = al.ArtistId
    JOIN tracks t ON al.AlbumId = t.AlbumId
    JOIN genres g ON t.GenreId = g.GenreId
)
SELECT
    ArtistName as "Исполнитель",
    GenreName as "Жанр",
    TrackName as "Самая длинная песня",
    Duration as "Длительность"
FROM ArtistGenreTracks
WHERE duration_rank = 1  -- Берем только самую длинную песню для каждого артиста в каждом жанре
ORDER BY ArtistName, GenreName
LIMIT 20;  -- Ограничим вывод для примера

 * sqlite:///store.db
Done.


Исполнитель,Жанр,Самая длинная песня,Длительность
AC/DC,Rock,Overdose,369319
Aaron Copland & London Symphony Orchestra,Classical,Fanfare for the Common Man,198064
Aaron Goldberg,Jazz,OAM's Blues,266936
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,Fantasia On Greensleeves,268066
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,Classical,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525: I. Allegro",348971
"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair",Classical,"Requiem, Op.48: 4. Pie Jesu",258924
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,"Suite No. 3 in D, BWV 1068: III. Gavotte I & II",225933
Accept,Rock,Princess of the Dawn,375418
Adrian Leaper & Doreen de Feis,Classical,"Symphony No. 3 Op. 36 for Orchestra and Soprano ""Symfonia Piesni Zalosnych"" \ Lento E Largo - Tranquillissimo",567494
Aerosmith,Rock,Livin' On The Edge,381231


## Задание 2
Посчитать скользящее среднее выручки по месяцам с шириной окна 3.


In [8]:
%%sql
WITH MonthlyRevenue AS (
    -- Вычисляем выручку по месяцам
    SELECT
        strftime('%Y-%m', i.InvoiceDate) as month,
        SUM(i.Total) as monthly_revenue
    FROM invoices i
    GROUP BY strftime('%Y-%m', i.InvoiceDate)
    ORDER BY month
)
SELECT
    month as "Месяц",
    monthly_revenue as "Выручка за месяц",
    ROUND(
        AVG(monthly_revenue) OVER (
            ORDER BY month
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ),
        2
    ) as "Скользящее среднее (окно 3)"
FROM MonthlyRevenue
ORDER BY month;

 * sqlite:///store.db
Done.


Месяц,Выручка за месяц,Скользящее среднее (окно 3)
2023-01,240.03000000000003,240.03
2023-02,311.19999999999993,275.62
2023-03,362.54,304.59
2023-04,179.39,284.38
2023-05,278.00999999999993,273.31
2023-06,180.70999999999998,212.7
2023-07,380.30999999999995,279.68
2023-08,315.32,292.11
2023-09,244.49999999999997,313.38
2023-10,267.02,275.61


## Задание 3
Составить рейтинг клиентов по количеству потраченных ими денег.

In [12]:
%%sql
SELECT
    c.CustomerId,
    c.FirstName || ' ' || c.LastName as CustomerName,
    SUM(i.Total) as TotalSpent,
    COUNT(i.InvoiceId) as PurchaseCount
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId, c.FirstName, c.LastName, c.Country, c.City
ORDER BY TotalSpent DESC;

 * sqlite:///store.db
Done.


CustomerId,CustomerName,TotalSpent,PurchaseCount
38,Niklas Schröder,116.28999999999999,7
42,Wyatt Girard,109.85,7
2,Leonie Köhler,108.33999999999999,7
3,François Tremblay,105.85999999999999,7
23,John Gordon,105.83,7
40,Dominique Lefebvre,103.33999999999999,7
37,Fynn Zimmermann,102.87,7
7,Astrid Gruber,102.38999999999999,7
57,Luis Rojas,100.91999999999999,7
59,Puja Srivastava,100.88,6


## Задание 4

Для каждого жанра вывести треки, которые встречались в большем числе заказов,
сумму, потраченную на данные треки, количество купленных треков и компанию, которая больше всех приобрела данную композицию.

In [17]:
%%sql
WITH TrackOrderStats AS (
    SELECT
        g.GenreId,
        g.Name as GenreName,
        t.TrackId,
        t.Name as TrackName,
        t.UnitPrice,
        ii.Quantity,
        i.InvoiceId,
        i.CustomerId,
        c.Company,
        t.UnitPrice * ii.Quantity as LineTotal
    FROM genres g
    JOIN tracks t ON g.GenreId = t.GenreId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    JOIN invoices i ON ii.InvoiceId = i.InvoiceId
    JOIN customers c ON i.CustomerId = c.CustomerId
),
GenreTrackSummary AS (
    SELECT
        GenreId,
        GenreName,
        TrackId,
        TrackName,
        COUNT(DISTINCT InvoiceId) as OrderCount,
        SUM(LineTotal) as TotalSpent,
        SUM(Quantity) as TotalQuantity
    FROM TrackOrderStats
    GROUP BY GenreId, GenreName, TrackId, TrackName
),
CompanyPurchases AS (
    SELECT
        TrackId,
        Company,
        SUM(Quantity) as CompanyQuantity,
        SUM(LineTotal) as CompanySpent,
        ROW_NUMBER() OVER (PARTITION BY TrackId ORDER BY SUM(Quantity) DESC) as CompanyRank
    FROM TrackOrderStats
    WHERE Company IS NOT NULL
    GROUP BY TrackId, Company
)
SELECT
    gts.GenreName as "Жанр",
    gts.TrackName as "Трек",
    gts.OrderCount as "Количество заказов",
    ROUND(gts.TotalSpent, 2) as "Сумма потрачена",
    gts.TotalQuantity as "Количество купленных копий",
    cp.Company as "Компания-лидер по покупкам",
    cp.CompanyQuantity as "Куплено этой компанией"
FROM GenreTrackSummary gts
LEFT JOIN CompanyPurchases cp ON gts.TrackId = cp.TrackId AND cp.CompanyRank = 1
WHERE gts.OrderCount > 0
  AND cp.Company IS NOT NULL
ORDER BY gts.GenreName, gts.OrderCount DESC, gts.TotalSpent DESC
LIMIT 20;

 * sqlite:///store.db
Done.


Жанр,Трек,Количество заказов,Сумма потрачена,Количество купленных копий,Компания-лидер по покупкам,Куплено этой компанией
Alternative,Scar On the Sky,1,2.97,3,Oracle,3
Alternative,Say Hello 2 Heaven,1,1.98,2,Oracle,2
Alternative,All Night Thing,1,1.98,2,Oracle,2
Alternative,Until We Fall,1,0.99,1,Oracle,1
Alternative,Show Me How to Live (Live at the Quart Festival),1,0.99,1,Oracle,1
Alternative,Slowness,1,0.99,1,BMW,1
Alternative & Punk,Caffeine,2,11.94,6,BMW,3
Alternative & Punk,Basket Case,2,7.96,4,Microsoft Corporation,3
Alternative & Punk,Bowels Of The Devil,2,5.98,2,Oracle,1
Alternative & Punk,When I Come Around,2,5.97,3,JetBrains s.r.o.,2


## Задание 5

Для каждого плейлиста вывести трек с наибольшей продолжительностью, трек с наименьшей стоимостью (если у треков одинаковая стоимость, то вывести тот, который весит меньше всех), и жанр данного плейлиста (жанром плейлиста будет называться тот жанр, треков которого больше всех в плейлисте).

In [19]:
%%sql
WITH PlaylistStats AS (
    SELECT
        p.PlaylistId,
        p.Name as PlaylistName
    FROM playlists p
)
SELECT
    ps.PlaylistName as "Плейлист",
    (SELECT t.Name
     FROM playlist_track pt
     JOIN tracks t ON pt.TrackId = t.TrackId
     WHERE pt.PlaylistId = ps.PlaylistId
     ORDER BY t.Milliseconds DESC, t.TrackId
     LIMIT 1) as "Самый длинный трек",

    (SELECT t.Name
     FROM playlist_track pt
     JOIN tracks t ON pt.TrackId = t.TrackId
     WHERE pt.PlaylistId = ps.PlaylistId
     ORDER BY t.UnitPrice ASC, t.Bytes ASC, t.TrackId
     LIMIT 1) as "Самый дешевый трек",

    (SELECT g.Name
     FROM playlist_track pt
     JOIN tracks t ON pt.TrackId = t.TrackId
     JOIN genres g ON t.GenreId = g.GenreId
     WHERE pt.PlaylistId = ps.PlaylistId
     GROUP BY g.GenreId, g.Name
     ORDER BY COUNT(*) DESC, g.GenreId
     LIMIT 1) as "Основной жанр"
FROM PlaylistStats ps
ORDER BY ps.PlaylistId;

 * sqlite:///store.db
Done.


Плейлист,Самый длинный трек,Самый дешевый трек,Основной жанр
From That TV Show,Occupation / Precipice,Your Time Is Gonna Come,TV Shows
Romantic Collection 87,Advance Romance,"Oh, My Love",Rock
90’s Music,Coma,Perfect Crime,Rock
Guitar Classic,None,None,None
Hidden Gems,Occupation / Precipice,Greatest Hits,TV Shows
Brazilian Music,Vai Passar,A Banda,Latin
Classical,"Symphony No. 3 Op. 36 for Orchestra and Soprano ""Symfonia Piesni Zalosnych"" \ Lento E Largo - Tranquillissimo","L'orfeo, Act 3, Sinfonia (Orchestra)",Classical
The Basics Of Classic,"Adagio for Strings from the String Quartet, Op. 11","Aria Mit 30 Veränderungen, BWV 988 ""Goldberg Variations"": Aria",Classical
Your Sisters' Favorite Tracks,Alive,American Idiot,Rock
Rock Classic,Master Of Puppets,Run to the Hills,Metal
